In [14]:
from pydantic import BaseModel
from rich import print
from rich.pretty import pprint
from icecream import ic
from typing import Optional
from datetime import date
from pydantic import Field
from pydantic import ValidationError

In [6]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

In [20]:
p = Person(first_name="Isaac", last_name="Hamid", age=78)
print(p)

Person(first_name='Isaac', last_name='Hamid', age=78)

In [14]:
try:
    Person(first_name = "hamid")
except ValidationError as ex:
    print(ex.json())


[{"type":"missing","loc":["last_name"],"msg":"Field 
required","input":{"first_name":"hamid"},"url":"https://errors.pydantic.dev/2.4/v/missing"},{"type":"missing","loc"
:["age"],"msg":"Field required","input":{"first_name":"hamid"},"url":"https://errors.pydantic.dev/2.4/v/missing"}]

In [30]:
class Person(BaseModel):
    first_name: str = None
    last_name: str
    # age: int | None = None
    # age: Optional[int] = None
    age: int = None

In [32]:
try:
   p = Person(first_name="Isaac", last_name="Hamid", age=78)
except ValidationError as ex:
    print(ex)


In [34]:
p.model_dump() # dump the model as dictionary

{'first_name': 'Isaac', 'last_name': 'Hamid', 'age': 78}

In [36]:
p.model_dump_json()

'{"first_name":"Isaac","last_name":"Hamid","age":78}'

In [37]:
p.model_dump(exclude = {"first_name"})

{'last_name': 'Hamid', 'age': 78}

In [41]:
p.model_dump_json(exclude = {"last_name"})

'{"first_name":"Isaac","age":78}'

In [43]:
from datetime import date

In [44]:
class Person(BaseModel):
    first_name: str = None
    last_name: str
    # age: int | None = None
    # age: Optional[int] = None
    age: int = None
    dob: date 
    

In [12]:
data = {
    "first_name": "Isaac",
    "last_name": "Newton",
    "dob": date(1643, 1, 4)
}

In [50]:
p = Person.model_validate(data) # deserializing a dict
print(p)

Person(first_name='Isaac', last_name='Newton', age=None, dob=datetime.date(1643, 1, 4))

In [52]:
json_data= '''
{
    "first_name": "Isaac",
    "last_name": "Newton",
    "dob": "1643-01-04"
}
'''

In [54]:
p = Person.model_validate_json(json_data) # deserializing a dict
print(p)

Person(first_name='Isaac', last_name='Newton', age=None, dob=datetime.date(1643, 1, 4))

In [9]:
class Person(BaseModel):
    first_name: str = Field(default=None, alias="firstName")
    last_name: str = Field(alias = "lastName")
    # age: int | None = None
    # age: Optional[int] = None
    age: int = None
    dob: date

In [15]:
try:
    p = Person.model_validate(data) # deserializing a dict
except ValidationError as ex:
    print(ex.json())

[{"type":"missing","loc":["lastName"],"msg":"Field 
required","input":{"first_name":"Isaac","last_name":"Newton","dob":"1643-01-04"},"url":"https://errors.pydantic.dev
/2.4/v/missing"}]

In [16]:
data2 = {
    "firstName": "Isaac",
    "lastName": "Newton",
    "dob": date(1643, 1, 4)
}

try:
    p = Person.model_validate(data2) # deserializing a dict
except ValidationError as ex:
    print(ex)

In [18]:
p.model_dump()

{'first_name': 'Isaac',
 'last_name': 'Newton',
 'age': None,
 'dob': datetime.date(1643, 1, 4)}

In [41]:
class Person(BaseModel):
    first_name: str = Field(default=None, alias="firstName")
    last_name: str = Field(alias = "lastName")
    # age: int | None = None
    # age: Optional[int] = None
    age: int = None
    dob: date

    class Config:
        populate_by_name = True
        extra = "allow"
        extra = "forbid"

In [22]:
Person(first_name = "Hamid", lastName = "Adesokan", dob = "1643-01-04")

Person(first_name=None, last_name='Adesokan', age=None, dob=datetime.date(1643, 1, 4))

In [24]:
Person.model_validate(data2)

Person(first_name='Isaac', last_name='Newton', age=None, dob=datetime.date(1643, 1, 4))

In [28]:
p = Person(first_name = "Hamid", last_name = "Adesokan", dob = "1643-01-04")
print(p)

Person(first_name='Hamid', last_name='Adesokan', age=None, dob=datetime.date(1643, 1, 4))

In [30]:
# p.model_dump(by_alias = True)
p.model_dump()

{'first_name': 'Hamid',
 'last_name': 'Adesokan',
 'age': None,
 'dob': datetime.date(1643, 1, 4)}

In [31]:
data_junk = {**data, "junk": "extra field"}
data_junk

{'first_name': 'Isaac',
 'last_name': 'Newton',
 'dob': datetime.date(1643, 1, 4),
 'junk': 'extra field'}

In [32]:
hasattr(p, "first_name")

True

In [42]:
p = Person(**data_junk)

ValidationError: 1 validation error for Person
junk
  Extra inputs are not permitted [type=extra_forbidden, input_value='extra field', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/extra_forbidden

In [37]:
Person.model_validate(data_junk)

Person(first_name='Isaac', last_name='Newton', age=None, dob=datetime.date(1643, 1, 4), junk='extra field')

In [40]:
p.model_dump()

{'first_name': 'Isaac',
 'last_name': 'Newton',
 'age': None,
 'dob': datetime.date(1643, 1, 4),
 'junk': 'extra field'}

In [43]:
def snake_to_camel_case(value: str) -> str:
    if not isinstance(value, str):
        raise ValueError("Value must be a string")
    words = value.split('_')
    value = "".join(word.title() for word in words if word )

    return f"{value[0].lower()}{value[1:]}"


In [45]:
snake_to_camel_case("first_name")

'firstName'

In [47]:
class CustomeBaseModel(BaseModel):
    class Config:
        alias_generator = snake_to_camel_case
        extra = "forbid"
        populate_by_name = True

class Person(CustomeBaseModel):
    first_name: str = None
    last_name: str 
    # age: int | None = None
    # age: Optional[int] = None
    age: int = None
    dob: date = None

In [52]:
p = Person(first_name="isaac", lastName="Israel", junk = "hjjj")
print(p)

ValidationError: 1 validation error for Person
junk
  Extra inputs are not permitted [type=extra_forbidden, input_value='hjjj', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/extra_forbidden

In [57]:
from pydantic import conint, constr

In [54]:
class Test(CustomeBaseModel):
    age: conint(gt=0, le=150)

In [68]:
class Test(CustomeBaseModel):
    first_name: str = None
    last_name: constr(strip_whitespace=True, strict=True, min_length = 2)

In [70]:
p = Test(first_name = "Hamid", last_name = 100)
print(p)

ValidationError: 1 validation error for Test
last_name
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.4/v/string_type

In [74]:
from pydantic import field_validator
class Test(CustomeBaseModel):
    hash_tag: str

    @field_validator('hash_tag')
    def validate_hash_tag(cls, value):
        if not value.startswith('#'):
            raise ValueError("hash tage must start with #")
        return value



In [75]:
t = Test(hash_tag = "#effff")
print(t)

Test(hash_tag='#effff')

In [76]:
t = Test(hash_tag = "effff")
print(t)

ValidationError: 1 validation error for Test
hash_tag
  Value error, hash tage must start with # [type=value_error, input_value='effff', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error

In [77]:
class Test(CustomeBaseModel):
    hash_tag: constr(strip_whitespace=True, min_length=2, to_lower=True)

    @field_validator('hash_tag')
    def validate_hash_tag(cls, value):
        if not value.startswith('#'):
            return f"#{value}"
        return value

In [79]:
t = Test(hash_tag = "GGGG")
print(t)

Test(hash_tag='#gggg')

In [88]:
from enum import Enum
from typing import List, Tuple, Union

class PolygonType(Enum):
    triangle = 3
    rectangle = 4
    pentagon = 5
    hexagon = 6

In [81]:
t = PolygonType.triangle

In [82]:
t.value, t.name

(3, 'triangle')

In [94]:
class PolygonModel(CustomeBaseModel):
    polygon_type: PolygonType
    vertices: List[Tuple[Union[int, float], Union[int, float]]]

    @field_validator('vertices')
    def validate_vertices(cls, value, values):
        polygon_type = values
        if polygon_type:
            num_vertices_required = polygon_type.value
            if len(value) != num_vertices_required:
                raise ValueError(f"For a {polygon_type.name}, excatly {polygon_type.value} vertices are required")
        return value
                



In [95]:
PolygonModel(polygon_type = PolygonType.triangle, vertices = [(1,1), (2,2), (3,3)])

AttributeError: 'pydantic_core._pydantic_core.ValidationInfo' object has no attribute 'value'